In [1]:
import numpy as np
import numpy.linalg as nla
import scipy.linalg as sla
import copy
import pandas as pd

**1. Линейная регрессия.**

Дорешаем одну старую задачу - про линейную регрессию.

In [2]:
from sklearn.linear_model import LinearRegression

Попробуем предсказать курс доллара.
Используем линейную регрессию.

In [3]:
x = [[16, 1], [15, 1], [14, 1], [13, 1], [10, 1], [9, 1]]  #даты
y = [76.9808, 75.6826, 77.2535, 77.5104, 77.1657, 77.1011] #курс

# Предсказание на сайте: 17.04 	75.5535
# y = a * x + b * 1

Обучаем линейную регрессию


In [4]:
model = LinearRegression()

model.fit(x, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Предсказываем:

In [5]:
model.predict([[17, 1]])

array([76.58513433])

А теперь проверим, что результат получается умножением на псевдообратную матрицу

In [6]:
w = sla.pinv(x)@y
w

array([-0.08733176, 78.06977425])

In [7]:
x@w

array([76.67246609, 76.75979785, 76.84712961, 76.93446137, 77.19645665,
       77.28378841])

In [8]:
[17,1]@w

76.58513433476395

*Сошлось!*

**2. Метод итераций**

In [9]:
A = [[10, 2, 3], 
     [2, 10, 3], 
     [2, 3, 10]]

b = [23, 31, 38]

# Точное решение  A x = b суть (1, 2, 3)

# Простой метод итераций - переходим к 
# x = P x + y

P = [[0, -0.2, -0.3], 
     [-0.2, 0, -0.3], 
     [-0.2, -0.3, 0]]

y = [2.3, 3.1, 3.8]

Итерационные процесс:

In [10]:
%precision 4

x0 = [0, 0, 0]
x = x0

steps = 15
for i in range(steps):
  x = y + np.dot(P, x)
  print(x) 

[2.3 3.1 3.8]
[0.54 1.5  2.41]
[1.277 2.269 3.242]
[0.8736 1.872  2.8639]
[1.0664 2.0661 3.0637]
[0.9677 1.9676 2.9669]
[1.0164 2.0164 3.0162]
[0.9919 1.9919 2.9918]
[1.0041 2.0041 3.0041]
[0.998 1.998 2.998]
[1.001 2.001 3.001]
[0.9995 1.9995 2.9995]
[1.0003 2.0003 3.0003]
[0.9999 1.9999 2.9999]
[1.0001 2.0001 3.0001]


Видно, что процесс быстро сходится к точному решению (1, 2, 3)

**3. Метод Зейделя**

Представляем A = L + U, где L нижнетреугольная, а U строго верхнетреугольная (т.е. у U нули на диагонали).

In [11]:
L = copy.deepcopy(A)
U = copy.deepcopy(A)

for i in range(len(A)):
  for j in range(len(A)):
    if i >= j:
      U[i][j] = 0

for i in range(len(A)):
  for j in range(len(A)):
    if i < j:
      L[i][j] = 0

print(L)
print(U)

[[10, 0, 0], [2, 10, 0], [2, 3, 10]]
[[0, 2, 3], [0, 0, 3], [0, 0, 0]]


Метод Зейделя. От исходного уравнения мы переходим к эквивалентному уравнению $$ Lx = b - Ux.$$ Итеративный процесс $$ L x^{(k+1)} = b - Ux ^{(k)}.$$ Фактически это метод простых итераций для $$ x = L^{-1} (b- Ux).$$

In [12]:
%precision 4

x0 = [0, 0, 0]
x = x0

steps = 15
for i in range(steps):
  x = np.dot(nla.inv(L), b - np.dot(U, x))
  print(x)

[2.3   2.64  2.548]
[1.0076 2.1341 2.9583]
[0.9857 2.0154 2.9982]
[0.9975 2.001  3.0002]
[0.9997 2.     3.0001]
[1. 2. 3.]
[1. 2. 3.]
[1. 2. 3.]
[1. 2. 3.]
[1. 2. 3.]
[1. 2. 3.]
[1. 2. 3.]
[1. 2. 3.]
[1. 2. 3.]
[1. 2. 3.]


Преимущество метода Зейделя - его можно реализовать так, что на каждом шаге мы работаем лишь с искомым вектором x и одной строкой матрицы $A$.

In [13]:
%precision 4

x0 = [0, 0, 0]
x = x0

steps = 15
for s in range(steps):
  for i in range(len(A)):
    a = A[i][i]
    x[i] = b[i] / a
    for j in range(len(A)): # change element i
      if i != j:
        x[i] -= (A[i][j] * x[j] )/a
  print(x)


[2.3, 2.64, 2.548]
[1.0075999999999998, 2.13408, 2.958256]
[0.9857071999999998, 2.0153817600000004, 2.9982440319999997]
[0.9974504383999998, 2.00103670272, 3.0001989015039996]
[0.9997329890047999, 1.9999937317478402, 3.000055282674688]
[0.9999846688480254, 1.9999864814279884, 3.000007121801998]
[1.0000005671738026, 1.9999977500246402, 3.0000005615578473]
[1.0000002815277176, 1.9999997752271024, 3.0000000111263256]
[1.0000000416166817, 1.9999999883387658, 2.999999995175034]
[1.0000000037797367, 2.000000000691543, 2.9999999990365893]
[1.0000000001507146, 2.0000000002588805, 2.999999999892193]
[0.9999999999805659, 2.0000000000362292, 2.999999999993018]
[0.9999999999948486, 2.0000000000031246, 3.0000000000000924]
[0.9999999999993472, 2.000000000000103, 3.0000000000000995]
[0.9999999999999494, 1.9999999999999802, 3.000000000000016]


**4. Самая влиятельная вершина.** 

Рассмотрим неотрицательную матрицу. В столбце j стоят вероятности перейти из вершины j в другие вершины. 

In [14]:
A = [[0, 1/2, 1/3, 1, 0], 
     [1, 0, 1/3, 0, 1/3], 
     [0, 1/2, 0, 0, 1/3],
     [0, 0, 0, 0, 1/3],
     [0, 0, 1/3, 0, 0]]

Начнём в вектор $$x^{(0)} = \left(\frac{1}{n}, \dots, \frac{1}{n}\right)$$ и применим метод итераций $$ x^{(n)} = A^n x^{(0)}.$$

In [15]:
x = [1/5, 1/5, 1/5, 1/5, 1/5]

for i in range(101):
  x = np.dot(A, x)
  if i % 10 == 0:
    print("На шаге {:} вектор x = {:}".format(i, x))


На шаге 0 вектор x = [0.3667 0.3333 0.1667 0.0667 0.0667]
На шаге 10 вектор x = [0.2919 0.3919 0.2184 0.0243 0.0735]
На шаге 20 вектор x = [0.2927 0.3902 0.2196 0.0244 0.0732]
На шаге 30 вектор x = [0.2927 0.3902 0.2195 0.0244 0.0732]
На шаге 40 вектор x = [0.2927 0.3902 0.2195 0.0244 0.0732]
На шаге 50 вектор x = [0.2927 0.3902 0.2195 0.0244 0.0732]
На шаге 60 вектор x = [0.2927 0.3902 0.2195 0.0244 0.0732]
На шаге 70 вектор x = [0.2927 0.3902 0.2195 0.0244 0.0732]
На шаге 80 вектор x = [0.2927 0.3902 0.2195 0.0244 0.0732]
На шаге 90 вектор x = [0.2927 0.3902 0.2195 0.0244 0.0732]
На шаге 100 вектор x = [0.2927 0.3902 0.2195 0.0244 0.0732]


Посмотрим на точное решение. Отметим, что одно из собственных значений матрицы A - это 1. Действительно: $$(1,1,...,1) A = (1,1,...,1)$$ ведь сумма вероятностей в каждом столбце равна 1.

In [16]:
v = [4, 16/3, 3, 1/3, 1] # собственный вектор 
np.dot(A, v) - v # собственное значение равно 1

array([0., 0., 0., 0., 0.])

Нормируем вектор так, чтобы получить вероятности.

In [17]:
v/np.sum(v)

array([0.2927, 0.3902, 0.2195, 0.0244, 0.0732])

Это вектор, к которому стремился метод итераций.